<h1><font color='blue'> 8E and 8F: Finding the Probability P(Y==1|X)</font></h1>

<h2><font color='Geen'> 8E: Implementing Decision Function of SVM RBF Kernel</font></h2>

<font face=' Comic Sans MS' size=3>After we train a kernel SVM model, we will be getting support vectors and their corresponsing coefficients $\alpha_{i}$

Check the documentation for better understanding of these attributes: 

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
<img src='https://i.imgur.com/K11msU4.png' width=500>

As a part of this assignment you will be implementing the ```decision_function()``` of kernel SVM, here decision_function() means based on the value return by ```decision_function()``` model will classify the data point either as positive or negative

Ex 1: In logistic regression After traning the models with the optimal weights $w$ we get, we will find the value $\frac{1}{1+\exp(-(wx+b))}$, if this value comes out to be < 0.5 we will mark it as negative class, else its positive class

Ex 2: In Linear SVM After traning the models with the optimal weights $w$ we get, we will find the value of $sign(wx+b)$, if this value comes out to be -ve we will mark it as negative class, else its positive class.

Similarly in Kernel SVM After traning the models with the coefficients $\alpha_{i}$ we get, we will find the value of 
$sign(\sum_{i=1}^{n}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here $K(x_{i},x_{q})$ is the RBF kernel. If this value comes out to be -ve we will mark $x_{q}$ as negative class, else its positive class.

RBF kernel is defined as: $K(x_{i},x_{q})$ = $exp(-\gamma ||x_{i} - x_{q}||^2)$

For better understanding check this link: https://scikit-learn.org/stable/modules/svm.html#svm-mathematical-formulation
</font>

## Task E

> 1. Split the data into $X_{train}$(60), $X_{cv}$(20), $X_{test}$(20)

> 2. Train $SVC(gamma=0.001, C=100.)$ on the ($X_{train}$, $y_{train}$)

> 3. Get the decision boundry values $f_{cv}$ on the $X_{cv}$ data  i.e. ` `$f_{cv}$ ```= decision_function(```$X_{cv}$```)```  <font color='red'>you need to implement this decision_function()</font>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
X, y = make_classification(n_samples=5000, n_features=5, n_redundant=2,n_classes=2, weights=[0.7], class_sep=0.7, random_state=15)

In [ ]:
from sklearn.model_selection import train_test_split
X_tr,X_test,y_tr,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_cv,y_train,y_cv = train_test_split(X_tr,y_tr,test_size=0.2,random_state=24)
print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)
print(X_test.shape, y_test.shape)

(3200, 5) (3200,)
(800, 5) (800,)
(1000, 5) (1000,)


### Pseudo code

clf = SVC(gamma=0.001, C=100.)<br>
clf.fit(Xtrain, ytrain)

<font color='green'>def</font> <font color='blue'>decision_function</font>(Xcv, ...): #use appropriate parameters <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'>for</font> a data point $x_q$ <font color='green'>in</font> Xcv: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='grey'>#write code to implement $(\sum_{i=1}^{\text{all the support vectors}}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here the values $y_i$, $\alpha_{i}$, and $intercept$ can be obtained from the trained model</font><br>
   <font color='green'>return</font> <font color='grey'><i># the decision_function output for all the data points in the Xcv</i></font>
    
fcv = decision_function(Xcv, ...)  <i># based on your requirement you can pass any other parameters </i>

<b>Note</b>: Make sure the values you get as fcv, should be equal to outputs of clf.decision_function(Xcv)


In [ ]:
gamma=0.001
clf = SVC(gamma=gamma, C=100)
clf.fit(X_train,y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
def K(xq):
    val = 0
    for alpha,xi in zip(clf.dual_coef_[0],clf.support_vectors_): #the dual_coef_[i] contains label[i]*alpha[i]
        val += alpha*np.exp(-gamma*np.linalg.norm(xi-xq)**2) 
    return val+clf.intercept_.item()

In [ ]:
def dec_fun(X_cv):
    fcv = []
    for xq in X_cv:
        fcv.append(K(xq))
    fcv = np.array(fcv)
    return fcv

In [ ]:
fcv = dec_fun(X_cv)

In [ ]:
print(fcv[5:10])

[-2.30501509  1.41770579 -0.43831055 -2.43467245  1.64358234]


In [ ]:
clf.decision_function(X_cv)[5:10]

array([-2.30501509,  1.41770579, -0.43831055, -2.43467245,  1.64358234])

We observe that the custom function gives same values as the inbuilt decision function.

<h2><font color='Geen'> 8F: Implementing Platt Scaling to find P(Y==1|X)</font></h2>

Check this <a href='https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a'>PDF</a>
<img src='https://i.imgur.com/CAMnVnh.png'>


## TASK F


> 4. Apply SGD algorithm with ($f_{cv}$, $y_{cv}$) and find the weight $W$ intercept $b$ ```Note: here our data is of one dimensional so we will have a one dimensional weight vector i.e W.shape (1,)``` 

> Note1: Don't forget to change the values of $y_{cv}$ as mentioned in the above image. you will calculate y+, y- based on data points in train data

> Note2: the Sklearn's SGD algorithm doesn't support the real valued outputs, you need to use the code that was done in the `'Logistic Regression with SGD and L2'` Assignment after modifying loss function, and use same parameters that used in that assignment.
<img src='https://i.imgur.com/zKYE9Oc.png'>
if Y[i] is 1, it will be replaced with y+ value else it will replaced with y- value

> 5. For a given data point from $X_{test}$, $P(Y=1|X) = \frac{1}{1+exp(-(W*f_{test}+ b))}$ where ` `$f_{test}$ ```= decision_function(```$X_{test}$```)```, W and b will be learned as metioned in the above step

In [ ]:
def initialize_weights(dim):
    ''' In this function, we will initialize our weights and bias'''
    #initialize the weights to zeros array of (dim,1) dimensions
    #you use zeros_like function to initialize zero, check this link https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros_like.html
    #initialize bias to zero
    w = np.zeros_like(dim)
    b = 0

    return w,b

In [ ]:
def sigmoid(z):
    ''' In this function, we will return sigmoid of z'''
    # compute sigmoid(z) and return

    return 1 / (1 + np.exp(-z))

In [ ]:
def logloss(w, b, x, y):
    '''In this function, we will compute log loss '''
    N = len(y)
    sum_log = 0
    for i in range(N):
        sum_log += y[i] * np.log10(sigmoid(np.dot(w, x[i]) + b)) + (1 - y[i]) * np.log10(1 - sigmoid(np.dot(w, x[i]) + b))
    return -1 * sum_log/N

In [ ]:
def gradient_dw(x,y,w,b,alpha,N):
    '''In this function, we will compute the gardient w.r.to w '''
    
    dw = x * (y - sigmoid(np.dot(w,x) + b) - (alpha / N) * w)
            
    return dw

In [ ]:
 def gradient_db(x,y,w,b):
     '''In this function, we will compute gradient w.r.to b '''

     db = y - sigmoid(np.dot(w,x) + b)
    
     return db

In [ ]:
## https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
def converter(y_cv):
    l = []
    unique, counts = np.unique(y_cv, return_counts=True)
    d = dict(zip(unique, counts))
    
    p_plus = (d[1] + 1) / (d[1] + 2)
    p_minus = 1 / (d[0] - 1)
    
    
    for i in y_cv:
        if i == 0:
            l.append(p_minus)
        else:
            l.append(p_plus)
    ycv = np.array(l)
    return ycv

In [ ]:
ycv = converter(y_cv)
print(len(ycv))

800


In [ ]:
def train(fcv,ycv,epochs,alpha,eta0):
    test_loss = []
    w, b = initialize_weights(fcv[0])
    for i in range(epochs):
        for j in range(N):
            dw = gradient_dw(fcv[j],ycv[j],w,b,alpha,N)
            db = gradient_db(fcv[j],ycv[j],w,b)
            w = w + (eta0 * dw)
            b = b + (eta0 * db)  
        loss = logloss(w,b,fcv,ycv)
        test_loss.append(loss)
    return w,b,test_loss

In [ ]:
alpha=0.0001
eta0=0.0001
N=len(fcv)
epochs=50
w,b,test_loss=train(fcv,ycv,epochs,alpha,eta0)

In [ ]:
print("Updated Weight = ",w," and updated intercept =",b)

Updated Weight =  1.135848495170214  and updated intercept = -0.09358559976954377


__Note: in the above algorithm, the steps 2, 4 might need hyper parameter tuning, To reduce the complexity of the assignment we are excluding the hyerparameter tuning part, but intrested students can try that__


If any one wants to try other calibration algorithm istonic regression also please check these tutorials

1. http://fa.bianp.net/blog/tag/scikit-learn.html#fn:1

2. https://drive.google.com/open?id=1MzmA7QaP58RDzocB0RBmRiWfl7Co_VJ7

3. https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a

4. https://stat.fandom.com/wiki/Isotonic_regression#Pool_Adjacent_Violators_Algorithm
